**Written by Paula T. Wang, October 2022**

Git link: https://github.com/medianeuroscience/flow_replication.git

# Preprocessing data using fMRIprep

To run this notebook:
- Make sure data is in BIDS format
- Use brainiak conda environment
- Run notebook on a tmux session due to long runtime of fMRIprep
- Store output in a specified directory (e.g., user-defined scratch folder)

In [2]:
# Load initial settings
%config Completer.use_jedi = False
# Get current user
import getpass
user = getpass.getuser()
out_path = '/srv/scratch/pw/data/ncof2/'
print('Running code as: ', user)
print('Storing output at: ',out_path)

%autosave 5

Running code as:  pw
Storing output at:  /srv/scratch/pw/data/ncof2/


Autosaving every 5 seconds


# Create fMRIprep Output Directory

The output of fMRIprep should be stored in a `derivatives` folder inside the data's corresponding BIDS directory.  
Assuming you already have a BIDS directory for the tutorial data in your scratch workspace (e.g., /srv/scratch/fhopp/bids), you should create a derivatives folder in there and pipe the output of fMRIprep to that folder.  

Let us set up the paths in the below cell:

In [3]:
import shutil

try:
    shutil.rmtree(out_path+'derivatives') 
except:
    print("Directory doesn't exist. Skipping")

Directory doesn't exist. Skipping


In [5]:
import os, stat

try:
    os.mkdir(out_path+'derivatives')
except:
    print("Directory already exists. Skipping.")
    
# Grant root write access to our output folder 
os.chmod(out_path+'derivatives', os.stat(out_path).st_mode | ((stat.S_IRWXU | stat.S_IRWXO)))

Directory already exists. Skipping.


# Specify fMRIprep Command

Commented fMRIprep command (do not actually run).   
For a detailed explanation, see the [documentation](https://fmriprep.org/en/stable/).  
Note that you should only edit mount paths (-v) until the ':'
```
docker run ---privileged=true -rm --tty \ # basic docker run command, note that we add --privileged=True
-v /srv/scratch/fhopp/bids/:/base:ro \ # path to BIDS folder
-v /srv/scratch/fhopp/bids/derivatives:/out \ # path to output folder
-v /home/fhopp/github/lab_docs/tutorials/fmri/fmriprep/license.txt:/opt/freesurfer/license.txt:ro \ # path to freesurfer license file
nipreps/fmriprep:latest \ # specify fMRIprep docker container
# everything below are now positional arguments to fMRIprep!
/base/ \ # bids data INSIDE container
/out/\ # output data INSIDE container
participant --participant-label 01 \ # participant label
--notrack --ignore fieldmaps --fs-no-reconall \ # dont track usage data, ignore fieldmaps, do not run surface reconstruction
--nprocs 8 --mem 12000 # performance specifications for fMRIprep 
```

***

Execute the full command.  
Note that this will take a few hours. 

In [14]:
%%writefile fmriprep.sh
#!/bin/sh
#SBATCH --job-name=fmriprep      
#SBATCH --partition=mnl-all
#SBATCH --exclude=easley,nash,turing,wood,babbage
#SBATCH --array=1-35
#SBATCH --mail-type=ALL         
#SBATCH --mail-user=paulawang@ucsb.edu
#SBATCH --ntasks-per-node=1     # number of tasks per node
#SBATCH --mem-per-cpu=4G
#SBATCH -o /home/pw/projects/thesis/code/1_preprocessing/slurm_logs/%x-%A-%a.out
#SBATCH -e /home/pw/projects/thesis/code/1_preprocessing/slurm_logs/%x-%A-%a.err

# Parse the participants.tsv file and extract one subject ID from the line corresponding to this SLURM task.                                                                                           
SUB="$(sed -n "$((${SLURM_ARRAY_TASK_ID} + 1))s/sub-\(\S*\)\>.*/\1/gp" /home/pw/projects/thesis/data/bids/participants.tsv)"

# Docker command
cmd="docker run --privileged=true --rm --tty \
-v /srv/scratch/pw/data/ncof2/:/base:ro \
-v /srv/scratch/pw/data/ncof2/derivatives:/out \
-v /home/pw/projects/thesis/code/1_preprocessing/license.txt:/opt/freesurfer/license.txt:ro \
nipreps/fmriprep:22.0.0 /base/ /out/ \
participant \
--participant-label "${SUB}" \
--output-spaces MNI152NLin6Asym \
--ignore fieldmaps \
--fs-no-reconall \
--resource-monitor \
--nthreads 4 \
--omp-nthreads 6 \
--mem-mb 25000 \
--write-graph"

# Setup done, run the command
echo "PATH=$PATH"
echo "SHELL=$SHELL"
echo Running job: ${SLURM_ARRAY_JOB_ID}
echo Running task: ${SLURM_ARRAY_TASK_ID}
echo Running subject: $SUB
echo Commandline: $cmd
echo Executing on the machine: $(hostname)

eval $cmd

Overwriting fmriprep.sh


In [15]:
!sbatch fmriprep.sh

Submitted batch job 35732


In [4]:
#!/bin/sh
#SBATCH --job-name=fmriprep
#SBATCH --partition=cluster
#SBATCH --exclude=ada,babbage
#SBATCH --array=0-35
#SBATCH --mail-type=ALL         
#SBATCH --mail-user=paulawang@ucsb.edu
#SBATCH --nodes=2                # node count
#SBATCH --ntasks-per-node=1     # number of tasks per node
#SBATCH --cpus-per-task=12        # cpu-cores per task (>1 if multi-threaded tasks)
#SBATCH --mem-per-cpu=4G
# Outputs ----------------------------------
#SBATCH -o /home/pw/projects/thesis/data/code/preprocessing/slurm_logs/%x-%A-%a.out
#SBATCH -e /home/pw/projects/thesis/data/code/preprocessing/slurm_logs/%x-%A-%a.err
# ------------------------------------------


---

In [ ]:
!docker run --privileged=true --rm --tty -v /home/pw/projects/thesis/data/bids:/base:ro -v /home/pw/projects/thesis/data/bids/derivatives:/out -v /home/pw/projects/thesis/code/1_preprocessing/license.txt:/opt/freesurfer/license.txt:ro nipreps/fmriprep:22.0.0 /base/ /out/ participant --participant-label 006 --output-spaces MNI152NLin6Asym --ignore fieldmaps --fs-no-reconall --resource-monitor --nprocs 8 --mem 12000 --write-graph 


You are using fMRIPrep-22.0.0, and a newer version of fMRIPrep is available: 22.0.2.
Please check out our documentation about how and when to upgrade:
https://fmriprep.readthedocs.io/en/latest/faq.html#upgrading
bids-validator@1.8.0

This dataset appears to be BIDS compatible.

        Summary:                  Available Tasks:        Available Modalities: 
        564 Files, 22.22GB        game                    MRI                   
        35 - Subjects                                                           
        1 - Session                                                             


	If you have any questions, please post on https://neurostars.org/tags/bids.

221018-00:47:33,658 nipype.workflow IMPORTANT:
	 Running fMRIPrep version 22.0.0

         License NOTICE ##################################################
         fMRIPrep 22.0.0
         Copyright 2022 The NiPreps Developers.
         
         This product includes software developed by
         the NiPreps Com